# Investigating Pango recombinant (X-) origins in the sc2ts ARG

In [ ]:
import collections

import sc2ts
import numpy as np
import tskit
import numpy as np
import tskit_arg_visualizer as argviz

import nb_utils

from IPython.display import HTML
HTML('<style type="text/css">.progress .progress-bar::after{content:"🦠";display:block;text-align:right;margin-top:-2px;}'
     '.progress .progress-bar {background-color: #BBBBFF}</style>')

In [ ]:
# Get the Viridian ARG
ts = nb_utils.load()
ti = sc2ts.TreeInfo(ts)
print(f"{ts.num_nodes} nodes, {ts.num_edges} edges, {ts.num_mutations} mutations, {np.sum(ts.nodes_flags & sc2ts.NODE_IS_RECOMBINANT != 0)} recombination events")

### List out the pango-X nodes

In [ ]:
pangoFullX = [p for p, s in ti.pango_lineage_samples.items() if p.startswith("X")]
print(f"{len(pangoFullX)} full pango-X lineages=", pangoFullX)
pangoX = [p for p in pangoFullX if "." not in p]
print(f"\n{len(pangoX)} main pango-X lineages=", pangoX)

In [ ]:
lineage_defining_muts = nb_utils.read_in_mutations("../data/consensus_mutations.json.bz2")

In [ ]:
# Find most recent RE node above all samples of each type
from tqdm.auto import tqdm
MRC_RE = {pango: (None, np.inf) for pango in pangoX}
recombination_nodes = set(np.where(ts.nodes_flags & sc2ts.NODE_IS_RECOMBINANT)[0])
nodes_time = ts.nodes_time
for tree in tqdm(ts.trees()):
    for x in pangoX:
        samples = ti.pango_lineage_samples[x]
        if len(samples) == 0:
            continue
        u = samples[0] if len(samples) == 1 else tree.mrca(*samples)
        while u not in recombination_nodes:
            u = tree.parent(u)
            if u == tskit.NULL:
                break
        if u != tskit.NULL and nodes_time[u] < MRC_RE[x][1]:
            MRC_RE[x] = (u, nodes_time[u])

In [ ]:
# This is a bit tedious, as we have to look at all samples in all trees
samples = {pango: set() for pango in pangoX}
for tree in tqdm(ts.trees()):
    for pango, (potential_re, _) in MRC_RE.items():
        if potential_re is not None:
            samples[pango].update(tree.samples(potential_re))
        

In [ ]:
pango_counts = {pango: collections.Counter() for pango in pangoX}
sample_to_pango = {}
for p, sample_ids in ti.pango_lineage_samples.items():
    for s in sample_ids:
        sample_to_pango[s] = p
for pango, sample_set in samples.items():
    for s in sample_set:
        pango_counts[pango][sample_to_pango[s]] += 1

# Seemingly missing from Viridian QCed data
pango_counts["XD"] = None
pango_counts["XK"] = None
pango_counts["XT"] = None
pango_counts["XV"] = None
pango_counts["XAB"] = None
pango_counts["XAH"] = None
pango_counts["XAK"] = None
pango_counts["XAQ"] = None
pango_counts["XAR"] = None
pango_counts["XAT"] = None
pango_counts["XAW"] = None
pango_counts["XAY"] = None
pango_counts["XBA"] = None
pango_counts["XBC"] = None
# Others past XBH not added here

In [ ]:
print(
    f"In the following {len(pango_counts)} pangoX lineages,",
    "ones with an identified RE node are marked with '*'", 
    "('**' where they are the dominant descendant X lineage)"
)

tot_pango_x_re = []
pango_x_nodes = collections.defaultdict(set)
for pango in sorted(pango_counts, key=lambda x: (len(x), x)):
    if len(ti.pango_lineage_samples[pango]) == 0:
        print("  ", pango, "not in dataset")
    else:
        counts = pango_counts[pango]
        tot = counts.total()
        p = counts[pango]
        most_common_X = None
        is_recomb = (p > 0 and p/tot > 0.001)
        mark = "  "
        if is_recomb:
            pango_x_nodes[MRC_RE[pango][0]].add(pango)
            most_common_X = max([x for x in counts if x.startswith("X")], key=lambda x: counts[x])
            mark = "* "
            if most_common_X == pango:
                tot_pango_x_re.append(MRC_RE[pango][0])
                mark = "**"
        print(mark, f"{pango} ({tot} descendants of RE node {MRC_RE[pango][0]}, {p} {pango})", counts.most_common(3))
print(len(pango_x_nodes),
      "total pango X recombinant origins of which",
      len(tot_pango_x_re),
      "include all descendants of the dominant group (excpetions: XM and XBB)")
print(
    set(pango_x_nodes.keys()) - set(tot_pango_x_re), pango_x_nodes)

# Analysis of each pango recombinant

In [ ]:
NodeReport = collections.namedtuple("NodeReport", nb_utils.NODE_REPORT_KEYS)

d3arg = argviz.D3ARG.from_ts(ts, progress=True)

In [ ]:
# Label all recombination nodes with "*"
nb_utils.set_sc2ts_labels_and_styles(d3arg, ts, add_strain_names=False)
d3arg.nodes.loc[(d3arg.nodes.flag & sc2ts.NODE_IS_RECOMBINANT) > 0, 'label'] = "*"
d3arg.nodes.loc[d3arg.nodes.id == 200039, 'label'] = "*DELTA*"
d3arg.nodes.loc[d3arg.nodes.id == 822854, 'label'] = "*BA.2*"
d3arg.nodes.loc[d3arg.nodes.id == 1189192, 'label'] = "*BA.5*"
d3arg.nodes.loc[d3arg.nodes.id == 1, 'label'] = "Wuhan"

In [ ]:
def load_pango_position(d3arg, pangos, issue_number=None):
    pangos = [pangos] if isinstance(pangos, str) else pangos
    display(HTML(f"<h3>{' / '.join(pangos)}</h3>"))
    info = f"{' + '.join([str(len(ti.pango_lineage_samples[pX])) for pX in pangos])} samples"
    if issue_number is not None:
        info += f': sc2ts-paper <a href="https://github.com/jeromekelleher/sc2ts-paper/issues/{issue_number}">issue #{issue_number}</a>'
    display(HTML(info))
    try:
        d = nb_utils.set_x_01_from_json(d3arg, f"Viridian-{'-'.join(pangos)}.json")
    except FileNotFoundError:
        nb_utils.clear_x_01(d3arg)
    return pangos

In [ ]:
pangos = load_pango_position(d3arg, "XA")
    
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    highlight_mutations=lineage_defining_muts.get_unique_mutations(pangos[0], parent_pangos=("B.1.1.7", "B.1.177.18")),
    highlight_nodes=True, height=800)

In [ ]:
pangos = load_pango_position(d3arg, ["XB"], 332)

# this has too many samples so we collapse some
exclude = np.array(list(ts.first().samples(223239)))
exclude = exclude[exclude != 223230]

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    list(np.setdiff1d([u for pX in pangos for u in ti.pango_lineage_samples[pX]], exclude)),
    highlight_mutations=lineage_defining_muts.get_unique_mutations(pangos[0], parent_pangos=["B.1.243"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, "XC")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XC", parent_pangos=["AY.29", "B.1.1.7"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, ["XE", "XH"], 337)

print("Some weirdness going on with deletions just on the LHS of the breakpoint")
print("The 2 recombination nodes to the bottom right may be spurious")

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX][:20]] + [1212052, 1177107],
    child_levels=0,
    parent_levels=5,
    highlight_mutations=lineage_defining_muts.get_unique_mutations(pangos[0], parent_pangos=["BA.1.17.2", "BA.2"]),
    highlight_nodes=True, height=800, width=1000)

print("Possible alignment problems with the deletion here?")

NodeReport(*ti.node_report(965353)).copying_pattern

In [ ]:
pangos = load_pango_position(d3arg, ["XF"])
print("Looks clean")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XF", parent_pangos=["AY.4", "BA.1"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, "XG")
print("Some dodgy reverted deletions on the LHS branch. We probably got the breakpoint wrong, and it should be to the LHS of 6513")

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XG", parent_pangos=["BA.1.17", "BA.2.9"]),
    highlight_nodes=True, height=800, width=1000)

print("Moving the breakpoint actually costs no extra mutations:")
NodeReport(*ti.node_report(1083412)).copying_pattern

In [ ]:
pangos = load_pango_position(d3arg, "XJ")

nb_utils.plot_sc2ts_subgraph(
    d3arg,
   [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + [1090786],
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XJ", parent_pangos=["BA.1.17.2", "BA.2"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, "XL", )

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XL", parent_pangos=["BA.1.17.2", "BA.2"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, ["XM", "XAL"], 284)
print("Multiple origins of XM, but one dominant one (node 1003220)")
colours = ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', '#DDCC77']  # from https://personal.sron.nl/~pault/

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    child_levels=0,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XM", parent_pangos=["BA.1.1", "BA.2"]),
    highlight_nodes={c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, pangos)},
    height=800, width=1000)


NodeReport(*ti.node_report(1003220)).copying_pattern

In [ ]:
pangos = load_pango_position(d3arg, "XN", 358)
print("No recombination node")

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=6,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XN", parent_pangos=["BA.2.23"]),
    highlight_nodes=True, height=800, width=1000, include_mutation_labels=True)

In [ ]:
pangos = load_pango_position(d3arg, "XP", 345)
print("Apparently some missing deletion, but I can't see it")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=12,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XP", parent_pangos=["BA.1.1"]),
    highlight_nodes=True, height=800, width=1000, include_mutation_labels=True)

In [ ]:
pangoX = ["XQ", "XR", "XU", "XAA", "XAG", "XAM"]
pangos = load_pango_position(d3arg, pangoX, 338)
print("We pick a maximum of 10 samples from each group to display, plus some extra BA.2 samples that appear nested")
extras1 = [1249828, 1228294, 1219946, 1197469, 1182958, 1182957, 1161394, 1146404, 1146405]
extras2 = [2521553, 1152676, 1150120, 2513694, 2477211, 2466117, 2448160, 2449100]
extras3 = [1126313, 2534274, 2534275, 1141965, 2508149, 1105611, 1142202, 1111753]

colours = ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377']  # from https://personal.sron.nl/~pault/
colours = ['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', '#DDCC77']  # from https://personal.sron.nl/~pault/


nb_utils.plot_sc2ts_subgraph(
    d3arg, 
    [u for pX in pangos for u in ti.pango_lineage_samples[pX][:10]] + extras1 + extras2 + extras3 + [1200258, 1158324],
    parent_levels=7,
    child_levels=0,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XQ", parent_pangos=["BA.1.1.15", "BA.2.9"]),
    highlight_nodes={c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, pangoX)},
    height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, "XS", 287)
print("Something weird here, as there are two recombination nodes only separated by a deletion")
print("Apparently some XS samples have the deletion, and some don't?")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=6,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XS", parent_pangos=["AY.103", "BA.1.1"]),
    include_mutation_labels=True,
    highlight_nodes=True, height=500, width=800)

In [ ]:
pangos = load_pango_position(d3arg, "XW")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=6,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XW", parent_pangos=["BA.1.1.15", "BA.2"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, "XY")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=6,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XY", parent_pangos=["BA.1.1", "BA.2"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
colours = ['#332288', '#88CCEE', '#44AA99', '#999933', '#DDCC77']  # from https://personal.sron.nl/~pault/

pangoX = ["XZ", "XAC", "XAD", "XAE", "XAP"]
pangos = load_pango_position(d3arg, pangoX, 339)
cmap = {c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, pangoX)}
extra_BA_2 = [964554, 2340545, 2372712, 1056883] #, 1192387, 1112147, 1145629]
cmap.update({'lightgrey':extra_BA_2})
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + extra_BA_2,
    parent_levels=9,
    highlight_nodes=cmap,
    height=800, width=1000)

NodeReport(*ti.node_report(964555)).copying_pattern

In [ ]:
pangos = load_pango_position(d3arg, ["XAF"], 360)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + [1177107],
    parent_levels=10,
    child_levels=10,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAF", parent_pangos=["BA.2"]),
    highlight_nodes=True, height=800, width=1000
)

NodeReport(*ti.node_report(1177107)).copying_pattern

In [ ]:
pangos = load_pango_position(d3arg, ["XAJ"], 352)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=16,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAJ", parent_pangos=["BA.2.12"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, ["XAN", "XAV"], 353)
cmap = {c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, pangos)}

nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=10,
    #highlight_mutations=lineage_defining_muts.get_unique_mutations("XAN", parent_pangos=["BA.5.1"]),
    highlight_nodes=cmap,
    #include_mutation_labels=True,
    height=600, width=600)

In [ ]:
pangos = load_pango_position(d3arg, ["XAS"], 340)
print("Complex: multiple origins, but main clade does not seem to be a recombinant. 2 single-sample clades are, however")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=4,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAS", parent_pangos=["BA.4"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
cmap = {c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, ["XAU", "XN"])}

pangos = load_pango_position(d3arg, ["XAU", "XN"], 348)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + [1137492, 887654],
    parent_levels=20, child_levels=0,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAU", parent_pangos=["BA.2"]),
    highlight_nodes=cmap, height=1000, width=1000)

In [ ]:
pangos = load_pango_position(d3arg, ["XAV"], 354)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]],
    parent_levels=10, child_levels=1,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAJ", parent_pangos=["BA.2"]),
    highlight_nodes=True, height=600, width=1000,
    include_mutation_labels=True,
)

In [ ]:
pangos = load_pango_position(d3arg, ["XAZ"], 356)
print("There are a lot of XAZ samples, but they all form a clade, so just pick the first 20 for viz")
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX][:20]],
    parent_levels=10, child_levels=0,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XAZ", parent_pangos=["BA.2"]),
    highlight_nodes=True, height=800, width=1000,
    include_mutation_labels=True
)

In [ ]:
#nb_utils.set_sc2ts_labels_and_styles(d3arg, ts, add_strain_names=True)
pangos = load_pango_position(d3arg, ["XBB"])
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [1408964, 1396838, 1404568, 1423196, 1398292, 2681617, 1409763],
    parent_levels=8,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XBB", parent_pangos=["BA.2.10", "BM.1.1.1"]),
    highlight_nodes=True, height=600, width=800)

In [ ]:
#nb_utils.set_sc2ts_labels_and_styles(d3arg, ts, add_strain_names=True)

pangos = load_pango_position(d3arg, ["XBE"], 351)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + [2661358],
    parent_levels=15,
    highlight_mutations=lineage_defining_muts.get_unique_mutations("XBE", parent_pangos=["BA.5.2"]),
    highlight_nodes=True, height=800, width=1000)

In [ ]:
colours = ['#332288', '#88CCEE', '#44AA99', '#999933', '#DDCC77']  # from https://personal.sron.nl/~pault/
pangoX = ["XBQ", "XBK", "XBK.1", "CJ.1.3"]
pangos = load_pango_position(d3arg, pangoX, 349)
nb_utils.plot_sc2ts_subgraph(
    d3arg,
    [u for pX in pangos for u in ti.pango_lineage_samples[pX]] + [1363939, 1342796],
    parent_levels=10,
    highlight_mutations=lineage_defining_muts.get_unique_mutations(
        "XBQ", parent_pangos=["BM.1.1.1", "BM.1.1", "BA.2", "BA.2.75", "BA.2.75.3"]
    ),
    highlight_nodes={c: ti.pango_lineage_samples[pX] for c, pX in zip(colours, pangoX)},
    height=800, width=1000)

## Extra notes below

In [ ]:
#df = ti.recombinants_summary()
#df.set_index("recombinant", inplace=True)


# Here we document some investigations into Pango X assignments that appear not to have re nodes
reasons_for_no_re = dict(
    XP="Dependent on a deletion which sc2ts does not use",
    XN="Definitely not a recombinant according to sc2ts. See https://github.com/jeromekelleher/sc2ts-paper/issues/285",
    XAU="Definitely not a recombinant according to sc2ts. See https://github.com/jeromekelleher/sc2ts-paper/issues/285",
    XAZ="Not a recombinant. Nearest RE node is #1189192 which is a BA.5 with 112911 descendants",
    XAJ="Probably not a recombinant. See https://github.com/jeromekelleher/sc2ts-paper/issues/285",
    XAS="Complex: multiple origins, but main clade does not seem to be a recombinant. 2 single-sample clades are, however",
    XAV="Possibly not a recombinant: see https://github.com/jeromekelleher/sc2ts-paper/issues/285",
    XBQ="Probably not a recombinant. See https://github.com/jeromekelleher/sc2ts-paper/issues/285",
    XBK="Probably not a recombinant. See https://github.com/jeromekelleher/sc2ts-paper/issues/285",
)

display(HTML("<h3>SUMMARY</h3>"))
display(HTML("<h4>PangoX with a valid RE node</h4>"))
display(HTML("<dl>"))

for pango in sorted(pango_counts, key=lambda x: (len(x), x)):
    if pango_counts[pango] is None:
        display(HTML(f"&nbsp;&nbsp;&nbsp;{pango}: No samples in Viridian which passed QC"))
        continue
    try:
        node = pango_x_to_node[pango]
        others = pango_x_nodes[node]
        #row = df[node]
        display(HTML(
            f"* {pango}: re_node {node}" +
            (f" ({others.index(pango) + 1}/{len(others)})" if len(others) > 1 else "") +
            f", {int(row.interval_left)}-{int(row.interval_right)}bp" +
            f", {row.parent_left_pango} + {row.parent_right_pango}"
        ))
    except KeyError:            
        display(HTML(f"&nbsp;&nbsp;&nbsp;{pango}: {reasons_for_no_re.get(pango, 'No clear associated re node')}"))
        
